In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from architectures import *
from torch.utils.tensorboard import SummaryWriter

# Test whether aggregation moduls have correct dimensionality:
$ f : \mathcal{P}\left( \mathbb{R}^{(B \times C \times H \times W)} \right) \rightarrow \mathbb{R}^{(B \times C \times H \times W)} $ 

In [3]:
try:
    del agg
except:
    pass

agg = AttentionAggregation().cuda()
S,B,C,H,W = 3, 2, 64, 256, 256    # Sequence length, Batch, Channels, Height, Width
x = torch.rand(B, S, C, H, W).cuda()
print(agg(x).size())
print("Success")
del agg

torch.Size([2, 64, 256, 256])
Success


In [12]:
torch.rand(B, S, C, H, W).permute(1,0,2,3,4).size()

torch.Size([5, 2, 64, 256, 256])

In [2]:
embed_dim = 64

encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=4, dim_feedforward=64)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=1).cuda()

src = torch.rand(5, 2*256**2, embed_dim).cuda()
out = transformer_encoder(src)
del transformer_encoder
out.size()

torch.Size([5, 131072, 64])

In [2]:
batch_size, features, length = 256**2, 64, 10 
multihead_attn = nn.MultiheadAttention(64,8).cuda()
x = torch.rand(length,batch_size, features).cuda()
attn_output, attn_output_weights = multihead_attn(x,x,x)


(attn_output_weights > 0).all()

tensor(True, device='cuda:0')

In [4]:
S,B,C,H,W = 11, 2, 64, 256, 256
xs = [torch.zeros(B,C,H,W) for _ in range(S)]
agg = AttentionAggregation(embed_dim=C)
xs = agg(xs)
xs.size()

torch.Size([11, 2, 64, 256, 256])

In [15]:
x = torch.rand(10,64,256,256)*100
w = torch.ones(10,64,256,256)
softm = nn.Softmax(0)
w = softm(w)
(x.mean(0) - (x*w).sum(0)).mean()

tensor(-8.7497e-07)